In [ ]:
import torch
import torch.nn as nn
import d2l.torch as d2l

## 加载数据

In [ ]:
batch_size = 512
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)

## 定义模型

In [ ]:
class VGGBlock(nn.Module):
    def __init__(self, num_convs, in_channels, out_channels):
        super().__init__()
        layers = []
        for i in range(num_convs):
            layers.append(nn.Conv2d(in_channels, out_channels, 3, padding=1))
            layers.append(nn.ReLU())
            in_channels = out_channels
        layers.append(nn.MaxPool2d(2, 2))
        self.block = nn.Sequential(*layers)

    def forward(self, X):
        Y = self.block(X)
        return Y

In [ ]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

class VGG(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = VGGBlock(1, 1, 64)  # [64, 112, 112]
        self.block2 = VGGBlock(1, 64, 128)  # [128, 56, 56]
        self.block3 = VGGBlock(2, 128, 256)  # [256, 28, 28]
        self.block4 = VGGBlock(2, 256, 512)  # [512, 14, 14]
        self.block5 = VGGBlock(2, 512, 512)  # [512, 7, 7]
        self.linear = nn.Sequential(
            nn.Flatten(),  # [512 * 7 * 7]
            nn.Linear(512 * 7 * 7, 4096),  # [4096]
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(4096, 4096),  # [4096]
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(4096, 10),  # [10]
        )

    def forward(self, X):
        X = X.reshape(-1, 1, 224, 224)
        X = self.block1(X)
        X = self.block2(X)
        X = self.block3(X)
        X = self.block4(X)
        X = self.block5(X)
        y = self.linear(X)
        return y

## 训练

In [ ]:
# 模型准确率评估函数
def evaluate_accuracy(model, test_iter, device):
    metrics = d2l.Accumulator(2)
    model.to(device)
    model.eval()
    for X, y in test_iter:
        X, y = X.to(device), y.to(device)
        y_hat = model(X)
        metrics.add(d2l.accuracy(y_hat, y), y.numel())
    return metrics[0] / metrics[1]

In [ ]:
# 训练函数
def train(model, train_iter, test_iter, loss_fn, optimizer, num_epochs, device):
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0, 1],
                            legend=['train_loss', 'train_acc', 'test_acc'])
    metrics = d2l.Accumulator(3)
    timer = d2l.Timer()
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        metrics.reset()
        timer.start()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = model(X)
            loss = loss_fn(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            with torch.no_grad():
                metrics.add(loss * y.numel(), d2l.accuracy(y_hat, y), y.numel())
        timer.stop()
        print(f'speed: {metrics[2] / timer.times[-1]:.1f} samples/sec')
        test_acc = evaluate_accuracy(model, test_iter, d2l.try_gpu())
        print(f'train_acc: {metrics[1] / metrics[2]:.3f}, test_acc: {test_acc}')
        animator.add(epoch + 1,
                     (metrics[0] / metrics[2], metrics[1] / metrics[2], test_acc))
    print(f'speed: {metrics[2] / timer.avg():.1f} samples/sec on {device}.')
    print(f'train_acc: {metrics[1] / metrics[2]:.3f}, test_acc: {test_acc}.')

In [ ]:
lr = 0.1
num_epochs = 10

model = VGG()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr)

train(model,  train_iter, test_iter, loss_fn, optimizer, num_epochs, d2l.try_gpu())